In [3]:
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


import warnings
warnings.filterwarnings("ignore")

# read data
df=pd.read_excel(io=r"your file path")
# print(df.values[0,0])
row = len(df.values)
# print(row)


FileNotFoundError: [Errno 2] No such file or directory: 'your file path'

In [4]:
# data spilit
features_num = 16
features = df.values[:, 0:features_num]
labels = df.values[:, features_num]
train_data, test_data, train_label, test_label = train_test_split(features, labels, random_state=0, train_size=0.8)

# Label coding
le = LabelEncoder()
y_train_encoded = le.fit_transform(train_label)
train_label_encoded = le.fit_transform(train_label)

# print("Encoded labels:", train_label_encoded)
# print(train_label)

NameError: name 'df' is not defined

In [5]:
# Sample weighting
class_weight = {0: 0.5, 1: 5.0, 2: 1.0, 3: 1.0, 4: 1.0}     # “balanced”
sample_weights = compute_sample_weight(class_weight, train_label_encoded)
# print("Sample weights:", sample_weights)

NameError: name 'train_label_encoded' is not defined

In [ ]:
'''Construct the SDM architecture'''
# Define base classifierS
tree = DecisionTreeClassifier(max_depth=5
                                    , criterion="entropy"
                                    , random_state=3
                                    , splitter="best"
                                    , min_samples_leaf=1
                                    , max_features = 14
                                    , min_samples_split = 16)
rf = RandomForestClassifier(n_estimators=40
                               , min_samples_split=5
                               , random_state=0
                               , oob_score=True
                               , criterion="entropy"
                               , max_depth=3)
lr = LogisticRegression(    
    penalty='l2',
    C=20,
    solver='lbfgs',
    multi_class='multinomial',
    max_iter=1000,
    class_weight='balanced')
svc = SVC(probability=True)
ada = AdaBoostClassifier(n_estimators=50)
gbc = GradientBoostingClassifier(n_estimators=50)
etc = ExtraTreesClassifier(n_estimators=10)

# Define a Soft vote classifier
model = VotingClassifier(estimators=[
    ('dt', tree),
    ('rf', rf),
    ('lr', lr),
    ('svc', svc),
    ('ada', ada),
    ('gbc', gbc),
    ('etc', etc),
    ],
    voting = 'soft', weights = [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.1])

In [ ]:
# train
model = model.fit(train_data, train_label, sample_weight=sample_weights)

score_test = model.score(test_data, test_label)
print('Test data accuracy: ', score_test)
score_train = model.score(train_data, train_label)
print('Train data accuracy: ', score_train)

#save model
joblib.dump(model, r"your file path")

In [6]:
# 10-fold cross-validation
from sklearn.model_selection import cross_val_score, KFold
kfold = KFold(n_splits=10)
scores = cross_val_score(model, features, labels, cv=kfold)
for i, score in enumerate(scores):
    print("Fold {}: {:.4f}".format(i+1, score))

print("Average Accuracy: {:.4f}".format(scores.mean()))

# F1_score
from sklearn.metrics import f1_score
pre_test_label = model.predict(test_data)
print("F1_score:", f1_score(test_label, pre_test_label, average='macro'))


In [7]:
# confusion matrix
disp = ConfusionMatrixDisplay.from_estimator(
        model,
        test_data,
        test_label,
        cmap=plt.cm.Blues,
        normalize="true",
    )
disp.ax_.set_title('WRF')
plt.savefig(r"The path where you save")
plt.show()


NameError: name 'test_data' is not defined